In [4]:
import sklearn # Check Sklearn version
sklearn.__version__

'1.6.1'

In [5]:
!python --version

Python 3.13.1


In [20]:
import sagemaker
sagemaker.__version__

'2.242.0'

In [32]:
!pip install flask==1.1.1

  Using cached Flask-1.1.1-py2.py3-none-any.whl.metadata (4.4 kB)
Using cached Flask-1.1.1-py2.py3-none-any.whl (94 kB)
  Attempting uninstall: flask
    Found existing installation: Flask 2.2.2
    Uninstalling Flask-2.2.2:
      Successfully uninstalled Flask-2.2.2


## 1. Initialize Boto3 SDK and create S3 bucket. 

In [2]:
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'mainbucketrockhight5461' # Mention the created S3 bucket name here
print("Using bucket " + bucket)
# hi
print(f"sagemaker version: {sagemaker.__version__}")

/home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/25/25 22:21:26] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=984423;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=58703;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/murivirg/.config/sagemaker/config.yaml


[03/25/25 22:21:28] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=623685;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=414488;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py#1278\1278]8;;\

Using bucket mainbucketrockhight5461
sagemaker version: 2.242.0


## 2. save the model.pkl into model.tar.gz

In [3]:
import tarfile

with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("model.pkl")

### 2.1 we will test the model locally

In [4]:
import pickle
import pandas as pd
from io import StringIO
import numpy as np

# Step 1: Load the model from model.pkl
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

# Step 2: Prepare test data
test_data = pd.DataFrame([
    [1454.0, 1.0, 0.5, 1.0, 1.0, 0.0, 34.0, 0.7, 83.0, 4.0, 3.0, 250.0, 1033.0, 3419.0, 7.0, 5.0, 5.0, 1.0, 1.0, 0.0],
    [1092.0, 1.0, 0.5, 1.0, 10.0, 0.0, 11.0, 0.5, 167.0, 3.0, 14.0, 468.0, 571.0, 737.0, 14.0, 4.0, 11.0, 0.0, 1.0, 0.0]
])

# Convert to CSV string (mimicking CSVSerializer)
csv_buffer = StringIO()
test_data.to_csv(csv_buffer, header=False, index=False)
csv_data = csv_buffer.getvalue()

# Step 3: Parse the CSV string back to a DataFrame (mimicking input_fn in predict.py)
df_input = pd.read_csv(StringIO(csv_data), header=None)

# Step 4: Make predictions using the loaded model
predictions = model.predict(df_input)

# Step 5: Convert predictions to NumPy array (mimicking NumpyDeserializer)
result = np.array(predictions)

# Step 6: Print the result
print("Predictions:", result)

Predictions: [3 0]


/home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


## 3. Store Model Artifacts(model.tar.gz) into the S3 Bucket. 

In [5]:
s3 = boto3.client('s3')

# Upload the tar.gz file to S3
s3.upload_file("model.tar.gz", bucket, "models/model.tar.gz")
model_data = f"s3://{bucket}/models/model.tar.gz"

print(f"model data: {model_data}")

model data: s3://mainbucketrockhight5461/models/model.tar.gz


# 4. now we crate a docker container and we test it locally
run:
`sh test.sh`
Note: this script will remove all your running docker containers

In [13]:
import requests

# Define the ping endpoint URL
ping_url = "http://localhost:8081/ping"

try:
    # Send GET request with a 5-second timeout
    response = requests.get(ping_url, timeout=5)
    response.raise_for_status()  # Raises an exception for 4xx/5xx status codes
    print("Ping successful, status code:", response.status_code)
except requests.RequestException as e:
    print("Ping failed:", e)

Ping successful, status code: 200


In [14]:
import pandas as pd
import requests
from io import StringIO

# Define test data (same as in your test script)
test_data = pd.DataFrame([
    [1454.0, 1.0, 0.5, 1.0, 1.0, 0.0, 34.0, 0.7, 83.0, 4.0, 3.0, 250.0, 1033.0, 3419.0, 7.0, 5.0, 5.0, 1.0, 1.0, 0.0],
    [1092.0, 1.0, 0.5, 1.0, 10.0, 0.0, 11.0, 0.5, 167.0, 3.0, 14.0, 468.0, 571.0, 737.0, 14.0, 4.0, 11.0, 0.0, 1.0, 0.0]
])

# Convert DataFrame to CSV string without headers or index
csv_data = test_data.to_csv(header=False, index=False)

# Define the invocations endpoint URL
invocations_url = "http://localhost:8081/invocations"

# Set headers for CSV content type
headers = {"Content-Type": "text/csv"}

try:
    # Send POST request with CSV data and a 5-second timeout
    response = requests.post(invocations_url, headers=headers, data=csv_data, timeout=5)
    response.raise_for_status()  # Raises an exception for 4xx/5xx status codes
    predictions = response.json()  # Parse the JSON response
    print("Predictions:", predictions)
except requests.RequestException as e:
    print("Invocations failed:", e)
    if "response" in locals():  # Check if response exists before accessing it
        print("Response content:", response.text)

Predictions: [3, 0]


## deploy the container and test it
great so the container is working. Now we deploy it
use `sh ecr_test.sh`

In [15]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.deserializers import JSONDeserializer
import pandas as pd
import json

# Get the SageMaker execution role (assumes this is run in a SageMaker notebook)
role = "arn:aws:iam::794038231401:role/service-role/SageMaker-ExecutionRole-20250103T203496"

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Specify your ECR image URI (replace with your actual URI)
image_uri = '794038231401.dkr.ecr.us-east-1.amazonaws.com/my-sagemaker-model:latest'

# Create a dummy model artifact (required by SageMaker, even if the model is in the container)
dummy_file = 'dummy.txt'
with open(dummy_file, 'w') as f:
    f.write('dummy')

In [16]:
import tarfile
with tarfile.open('dummy.tar.gz', 'w:gz') as tar:
    tar.add('dummy.txt')

In [17]:
# Upload the dummy file to S3
def_bucket = sagemaker_session.default_bucket()
def_bucket = sagemaker_session.default_bucket()
print(def_bucket)
model_data = sagemaker_session.upload_data(
    path='model.tar.gz', 
    bucket=def_bucket, 
    key_prefix='model')


sagemaker-us-east-1-794038231401


In [18]:
print(model_data)

s3://sagemaker-us-east-1-794038231401/model/model.tar.gz


In [19]:
# Create the SageMaker model
model = Model(
    image_uri=image_uri,
    model_data=model_data,
    role=role,
    sagemaker_session=sagemaker_session
)

# Optional: Delete the endpoint after testing to avoid costs
# predictor.delete_endpoint()

In [20]:
# Deploy the model to an endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',  # Adjust instance type as needed
    endpoint_name='my-endpoint'   # Replace with a unique endpoint name
)

[03/25/25 22:52:56] INFO     Creating model with name: my-sagemaker-model-2025-03-26-04-52-56-805   ]8;id=807112;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=596213;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/25/25 22:52:57] INFO     Creating endpoint-config with name my-endpoint                         ]8;id=141577;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=761787;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py#5937\5937]8;;\

                    INFO     Creating endpoint with name my-endpoint                                ]8;id=611211;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=974615;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py#4759\4759]8;;\

--------!

In [21]:
print(type(predictor))

<class 'NoneType'>


In [22]:
import pandas as pd
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.deserializers import JSONDeserializer

# Create predictor object for existing endpoint
predictor = Predictor(endpoint_name='my-endpoint', sagemaker_session=sagemaker.Session())
predictor.deserializer = JSONDeserializer()

# Test data
test_data = pd.DataFrame([
    [1454.0, 1.0, 0.5, 1.0, 1.0, 0.0, 34.0, 0.7, 83.0, 4.0, 3.0, 250.0, 1033.0, 3419.0, 7.0, 5.0, 5.0, 1.0, 1.0, 0.0],
    [1092.0, 1.0, 0.5, 1.0, 10.0, 0.0, 11.0, 0.5, 167.0, 3.0, 14.0, 468.0, 571.0, 737.0, 14.0, 4.0, 11.0, 0.0, 1.0, 0.0]
])
csv_data = test_data.to_csv(header=False, index=False)

# Make prediction
try:
    response = predictor.predict(csv_data, initial_args={'ContentType': 'text/csv'})
    print("Predictions:", response)
except Exception as e:
    print("Prediction failed:", e)

Predictions: [3, 0]
